In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# PLUTO

In [18]:
colNames18 = ['borough','block', 'lot', 'ct2010',
            'zonedist1', 'overlay1', 'spdist1', 'ownertype',
            'bldgclass', 'landuse', 'lotarea', 'bldgarea', 'resarea',
            'numbldgs', 'numfloors', 'unitsres', 'unitstotal', 'bsmtcode',
            'assesstot', 'yearbuilt', 'yearalter1', 'yearalter2', 'builtfar',
            'residfar', 'borocode', 'bbl', 'tract2010', 'xcoord', 'ycoord', ]

dtypes18  = {'borough': str,'block': str, 'lot': str, 'cd': str, 'ct2010': str,
        'borocode': str, 'bbl': str, 'tract2010': str}


pluto18 = pd.read_csv('pluto18.csv', usecols = colNames18, dtype = dtypes18)
pluto18['year'] = '2018'


filenames = []
years = ['2014', '2015', '2016', '2017']
fileends = ['14.csv', '15.csv', '16.csv', '17.csv']
boros = ['BK', 'BX', 'MN', 'QN', 'SI']

for boro in boros:
    for fe in fileends:
        filenames.append(boro + fe)

colNames = ['Borough', 'Block', 'Lot', 'CT2010',
            'ZoneDist1','Overlay1', 'SPDist1', 'BldgClass', 'LandUse',
            'OwnerType', 'LotArea', 'BldgArea', 'ResArea',
            'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'BsmtCode',
            'AssessTot', 'YearBuilt', 'YearAlter1', 'YearAlter2',
            'BuiltFAR', 'ResidFAR', 'BoroCode', 'BBL', 'Tract2010', 'XCoord', 'YCoord']

dtypes = {'Borough': str, 'Block': str, 'Lot': str, 'CT2010': str,
        'BoroCode': str, 'BBL': str, 'Tract2010': str}


plutoMerge = pd.read_csv('pluto/BK14.csv',
                         usecols = colNames,
                         dtype = dtypes)
plutoMerge['year'] = '2014'

for file in filenames:
    if file == 'BK14.csv':
        continue
    else:
        newPluto = pd.read_csv('pluto/{}'.format(file),
                               usecols = colNames,
                               dtype = dtypes)
        newPluto['year'] = '20' + file[2:4]
        plutoMerge = plutoMerge.append(newPluto)
        
plutoMerge.columns = map(str.lower, plutoMerge.columns)

pluto = plutoMerge.append(pluto18).reset_index(drop = True)
pluto['ownertype'] = pluto.ownertype.fillna('P')
pluto['ct2010'] = pluto.ct2010.str.replace('.', '').str.pad(6, fillchar = '0')

county = ['047', '005', '061', '081', '085']
pluto['county'] = pluto['borough']

for i in range(5):
    pluto['county'] = pluto.county.str.replace(boros[i], county[i])
    
pluto['tract'] = '36' + pluto.county + pluto.ct2010

pluto.head()

,borough,block,lot,ct2010,zonedist1,overlay1,spdist1,bldgclass,landuse,ownertype,...,builtfar,residfar,borocode,bbl,tract2010,xcoord,ycoord,year,county,tract
0,BK,1,1,000021,M3-1,NaN,MX-2,V1,11,P,...,0.00,0.0,3,3000010001,21,987723,196222,2014,047,36047000021
1,BK,1,2,000021,M3-1,NaN,NaN,D7,4,P,...,12.81,0.0,3,3000010002,21,NaN,NaN,2014,047,36047000021
2,BK,1,50,000021,M1-4/R8A,NaN,MX-2,E9,6,P,...,7.84,5.4,3,3000010050,21,987838,195989,2014,047,36047000021
3,BK,3,1,000021,M3-1,NaN,NaN,U4,7,P,...,0.00,0.0,3,3000030001,21,988568,196151,2014,047,36047000021
4,BK,3,5,000021,M3-1,NaN,NaN,T2,7,C,...,0.00,0.0,3,3000030005,21,NaN,NaN,2014,047,36047000021


# Housing Violations

In [20]:
viol = pd.read_csv('housing_violations.csv', dtype = {'BBL': str})
viol.columns = map(str.lower, viol.columns)
viol = viol[viol.bbl != '0']
viol[['month', 'day','year']] = viol.inspectiondate.str.split('/', expand = True)
viol = viol[viol.year.astype(int) >= 2014]
viol = viol[viol['class'] != 'I']

In [21]:
viol.head()

,violationid,buildingid,registrationid,boroid,borough,housenumber,lowhousenumber,highhousenumber,streetname,streetcode,...,longitude,communityboard,councildistrict,censustract,bin,bbl,nta,month,day,year
0,10631871,324119,327612,3,BROOKLYN,261,261,261,LENOX ROAD,55330,...,-73.950946,17.0,40.0,820.0,3116276.0,3050660071,Prospect Lefferts Gardens-Wingate,03,16,2015
1,10631872,633260,408411,4,QUEENS,462,462,462,BEACH 67 STREET,31890,...,-73.796278,14.0,31.0,964.0,4302824.0,4160400030,Hammels-Arverne-Edgemere,03,16,2015
2,10631874,287536,305429,3,BROOKLYN,1159,1159,1159,EASTERN PARKWAY,38430,...,-73.929902,8.0,41.0,349.0,3324649.0,3013910061,Crown Heights North,03,16,2015
3,10631875,324119,327612,3,BROOKLYN,261,261,261,LENOX ROAD,55330,...,-73.950946,17.0,40.0,820.0,3116276.0,3050660071,Prospect Lefferts Gardens-Wingate,03,16,2015
4,10631876,100029,0,2,BRONX,1855,1855,1855,OBRIEN AVENUE,55020,...,-73.858742,9.0,18.0,2.0,2092578.0,2034710001,Soundview-Castle Hill-Clason Point-Harding Park,03,10,2015


## Violation Counts by BBL

In [22]:
viol_counts = pd.DataFrame(viol.groupby(['bbl','year'])['violationid'].count()) \
                               .rename(columns = {'violationid': 'violations'}) \
                               .reset_index()

In [23]:
viol_counts.head()

,bbl,year,violations
0,1000077501,2017,19
1,1000077501,2018,2
2,1000157501,2014,1
3,1000157501,2017,6
4,1000157501,2018,1


In [24]:
data = pluto.merge(viol_counts, how = 'left', on = ['bbl', 'year'])
data['violations'] = data.violations.fillna(0)

In [25]:
data.head()

,borough,block,lot,ct2010,zonedist1,overlay1,spdist1,bldgclass,landuse,ownertype,...,residfar,borocode,bbl,tract2010,xcoord,ycoord,year,county,tract,violations
0,BK,1,1,000021,M3-1,NaN,MX-2,V1,11,P,...,0.0,3,3000010001,21,987723,196222,2014,047,36047000021,0.0
1,BK,1,2,000021,M3-1,NaN,NaN,D7,4,P,...,0.0,3,3000010002,21,NaN,NaN,2014,047,36047000021,0.0
2,BK,1,50,000021,M1-4/R8A,NaN,MX-2,E9,6,P,...,5.4,3,3000010050,21,987838,195989,2014,047,36047000021,0.0
3,BK,3,1,000021,M3-1,NaN,NaN,U4,7,P,...,0.0,3,3000030001,21,988568,196151,2014,047,36047000021,0.0
4,BK,3,5,000021,M3-1,NaN,NaN,T2,7,C,...,0.0,3,3000030005,21,NaN,NaN,2014,047,36047000021,0.0


In [26]:
data.to_csv('bbl_violations.csv', index = False)

# HPD Building Features

In [27]:
colNames = ['BuildingID', 'BoroID', 'Block', 'Lot', 'BIN', 'LegalStories', 'LegalClassA', 'LegalClassB', 'LifeCycle']
dtype = {'BuildingID': str, 'BoroID': str, 'Block': str, 'Lot': str, 'BIN': str}

hpd = pd.read_csv('hpd_buildings.csv',
                  usecols = colNames, dtype = dtype).dropna()

hpd['LegalStories'] = hpd.LegalStories.astype(int)
hpd['LegalClassA'] = hpd.LegalClassA.astype(int)
hpd['LegalClassB'] = hpd.LegalClassB.astype(int)

hpd['BBL'] = hpd.BoroID + hpd.Block.str.pad(5, fillchar =  '0') + hpd.Lot.str.pad(4, fillchar = '0')

hpd = hpd.drop(columns = ['BoroID', 'Block', 'Lot'])

hpd.columns = map(str.lower, hpd.columns)

hpd.head()

,buildingid,bin,legalstories,legalclassa,legalclassb,lifecycle,bbl
2,510665,4231778,2,1,0,Building,4108400001
3,510671,4232145,2,2,0,Building,4108600044
4,510672,4232146,2,2,0,Building,4108600047
5,510673,4231789,2,1,0,Building,4108400031
6,510674,4231804,2,1,0,Building,4108410034


In [28]:
data = data.merge(hpd, on = 'bbl', how = 'left')
data.head()

,borough,block,lot,ct2010,zonedist1,overlay1,spdist1,bldgclass,landuse,ownertype,...,year,county,tract,violations,buildingid,bin,legalstories,legalclassa,legalclassb,lifecycle
0,BK,1,1,000021,M3-1,NaN,MX-2,V1,11,P,...,2014,047,36047000021,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,BK,1,2,000021,M3-1,NaN,NaN,D7,4,P,...,2014,047,36047000021,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,BK,1,50,000021,M1-4/R8A,NaN,MX-2,E9,6,P,...,2014,047,36047000021,0.0,316596,3000002,9.0,0.0,0.0,Building
3,BK,3,1,000021,M3-1,NaN,NaN,U4,7,P,...,2014,047,36047000021,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,BK,3,5,000021,M3-1,NaN,NaN,T2,7,C,...,2014,047,36047000021,0.0,NaN,NaN,NaN,NaN,NaN,NaN


# Subsidies & Affordability Flags

In [34]:
dtype = {'bbl': str, 'buildingid': str, 'bin': str, 'year': str}

data = pd.read_csv('main.csv', dtype = dtype)
subBBL = pd.read_csv('subsidized_BBL.csv')
subBBL.columns = map(str.lower, subBBL.columns)
subBBL.year = subBBL.year.astype(str)
subBBL.bin = subBBL.bin.astype(str)
subBBL.bbl = subBBL.bbl.astype(str)
aff = pd.read_csv('affordability_clean.csv')
aff.columns = map(str.lower, aff.columns)
aff.year = aff.year.astype(str)
aff.bin = aff.bin.astype(str)
aff.bbl = aff.bbl.astype(str)

data = pd.merge(data, subBBL, how = 'left', left_on = ['bbl', 'bin', 'year'], right_on = ['bbl', 'bin', 'year'])
data = pd.merge(data, aff, how = 'left', left_on = ['bbl', 'bin', 'year'], right_on = ['bbl', 'bin', 'year'])

data = data[data.bldgclass != 'R0']
data['bldgclass'] = data.bldgclass.str.replace('[0-9]', '')
data = data[data.bldgclass.isin(['A', 'B', 'C', 'D', 'R', 'S'])]
data = data[data.bldgclass.isna() == False]
data = data[data.zonedist1.isin(['BPC', 'PARK']) == False]
data['zonedist1'] = data.zonedist1.str[0]
data['landuse'] = data.landuse.astype(float) - 1
data['borocode'] = data.borocode.astype(int) - 1
data['affordability_flag'][data['affordability_flag'].isna()] = 0

#times altered
data['alter'] = 0
data['alter'][data.yearalter1 > 0] = 1
data['alter'][data.yearalter2 > 0] = 2

data['subsidies_flag'] = data.subsidies_flag.fillna(0)

data.head()

C:\Users\adley\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\adley\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\adley\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,borough,block,lot,ct2010,zonedist1,overlay1,spdist1,bldgclass,landuse,ownertype,...,violations,buildingid,bin,legalstories,legalclassa,legalclassb,lifecycle,subsidies_flag,affordability_flag,alter
1,BK,1.0,2.0,21,M,NaN,NaN,D,3.0,P,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1
25,BK,20.0,6.0,21,M,NaN,MX-2,C,3.0,P,...,0.0,316605,3000012,5.0,23.0,0.0,Building,0.0,0.0,2
41,BK,26.0,38.0,21,M,NaN,NaN,S,3.0,P,...,0.0,389036,3000021,4.0,3.0,0.0,Building,0.0,0.0,1
45,BK,27.0,40.0,21,C,NaN,NaN,D,3.0,P,...,0.0,949339,3344163,7.0,94.0,0.0,Building,0.0,0.0,1
47,BK,28.0,5.0,21,C,NaN,NaN,D,3.0,P,...,1.0,216461,3329408,8.0,106.0,0.0,Building,1.0,0.0,1


In [39]:
dataFinal = data[['zonedist1', 'bldgclass', 'landuse', 'ownertype', 'lotarea', 'bldgarea',
       'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'bsmtcode', 'assesstot', 'yearbuilt', 'builtfar', 'residfar',
       'borocode', 'bbl', 'year', 'violations', 'legalstories', 'legalclassa',
       'legalclassb', 'subsidies_flag', 'affordability_flag', 'alter']]
dataFinal['yearbuilt'] = dataFinal.yearbuilt.astype(int)
dataFinal.to_csv('violfinal.csv')

C:\Users\adley\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Reduced Dataset

In [177]:
dataSmall = data.drop(['block', 'lot', 'ct2010', 'overlay1', 'spdist1',
                       'yearalter1', 'yearalter2', 'borough', 'tract2010', 'lifecycle',
                       'xcoord', 'ycoord', 'county', 'tract', 'buildingid', 'bin'], axis = 1)

In [187]:
cutCol = ['lotarea', 'bldgarea', 'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
          'assesstot', 'yearbuilt', 'builtfar', 'residfar', 'legalstories', 'legalclassa', 'legalclassb']


for col in cutCol:
    dataSmall[col] = pd.cut(dataSmall[col], 5, labels = ['0', '1', '2', '3', '4'])

In [178]:
dataSmall.describe()

,landuse,lotarea,bldgarea,resarea,numbldgs,numfloors,unitsres,unitstotal,bsmtcode,assesstot,...,builtfar,residfar,borocode,violations,legalstories,legalclassa,legalclassb,subsidies_flag,affordability_flag,alter
count,1.013944e+06,1.013801e+06,1.013944e+06,1.010205e+06,1.013801e+06,1.013801e+06,1.013944e+06,1.013944e+06,1.013944e+06,1.013944e+06,...,1.012616e+06,1.013944e+06,1.013944e+06,1.013944e+06,400263.000000,400263.000000,400263.000000,1.013944e+06,1.013944e+06,1.013944e+06
mean,4.383201e-01,7.138666e+03,9.482978e+03,9.113744e+03,1.642000e+00,2.445195e+00,9.335758e+00,9.493672e+00,2.088701e+00,3.011344e+05,...,1.239293e+00,1.387827e+00,2.374856e+00,9.270591e-01,2.925999,7.816548,0.134112,7.318945e-03,3.696457e-03,1.043174e-01
std,8.243647e-01,9.974930e+04,1.521649e+05,1.480641e+05,1.159892e+01,1.554355e+00,1.405134e+02,1.409712e+02,1.234782e+00,5.730003e+06,...,4.682237e+01,1.121667e+00,8.890411e-01,1.123410e+01,1.979713,23.487896,2.472948,8.523723e-02,1.208818e-01,3.442009e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.000000e+03,1.515000e+03,1.392000e+03,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.311200e+04,...,5.500000e-01,6.000000e-01,2.000000e+00,0.000000e+00,2.000000,2.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,2.500000e+03,2.128000e+03,1.982000e+03,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.985900e+04,...,8.400000e-01,9.000000e-01,2.000000e+00,0.000000e+00,3.000000,3.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,3.709000e+03,3.087000e+03,2.808000e+03,2.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,2.000000e+00,4.182300e+04,...,1.360000e+00,2.000000e+00,3.000000e+00,0.000000e+00,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
max,4.000000e+00,1.225343e+07,1.354011e+07,1.321140e+07,1.851000e+03,2.050000e+02,1.091400e+04,1.094800e+04,5.000000e+00,7.775024e+08,...,4.580100e+04,1.200000e+01,4.000000e+00,8.800000e+02,64.000000,2017.000000,652.000000,1.000000e+00,4.000000e+00,2.000000e+00


In [104]:
hasvio = data[data.violations > 0]
hasvio.to_csv('has_violations.csv', index = False)

In [40]:
bin_bbl_year = data[['bin', 'bbl', 'year']]
bin_bbl_year.to_csv('bin_bbl_year.csv', index = False)

In [214]:
data.to_csv('violfinal.csv', index = False)

In [9]:
yearCoords = data[['year', 'bin', 'bbl', 'xcoord', 'ycoord']]
yearCoords.head()

,year,bin,bbl,xcoord,ycoord
0,2014,NaN,3.000010e+09,987723,196222
1,2014,NaN,3.000010e+09,NaN,NaN
2,2014,3000002.0,3.000010e+09,987838,195989
3,2014,NaN,3.000030e+09,988568,196151
4,2014,NaN,3.000030e+09,NaN,NaN


In [10]:
yearCoords.to_csv('year_bin_bbl_coords.csv', index = False)

# Summary

In [21]:
summ1 = data.groupby('year')[['bin', 'bbl']].count()
summ2 = data.groupby('year')[['violations', 'unitsres']].sum()
summ = summ1.merge(summ2, left_on = summ1.index, right_on = summ2.index).rename(columns={'key_0':'year'})

,year,bin,bbl,violations,unitsres
0,2014,305264,873144,525649.0,8545670.0
1,2015,305405,873706,581786.0,8561911.0
2,2016,125465,863052,188461.0,4223632.0
3,2017,305172,873449,657967.0,8343235.0
4,2018,304597,873151,807286.0,8365916.0


In [27]:
summ.to_csv('summary_stats1.csv', index = False)